## Prepare environment

In [ ]:
from google.colab import drive

drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# clone repo via ssh

!cp /content/drive/MyDrive/.ssh /root/  -r
!chmod 644 /root/.ssh/known_hosts
!chmod 600 /root/.ssh/id_rsa
!ssh -T git@github.com

!git clone -q git@github.com:annapanfil/punctuation_prediction.git

Hi annapanfil/punctuation_prediction! You've successfully authenticated, but GitHub does not provide shell access.
Cloning into 'punctuation_prediction'...
remote: Enumerating objects: 85, done.
remote: Counting objects: 100% (85/85), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 85 (delta 28), reused 70 (delta 17), pack-reused 0
Receiving objects: 100% (85/85), 6.18 MiB | 4.52 MiB/s, done.
Resolving deltas: 100% (28/28), done.


In [ ]:
%cd punctuation_prediction/
!pip install -q -r requirements.txt

/content/punctuation_prediction
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 59.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 95.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.1 MB/s

In [ ]:
import os
os.environ['MLFLOW_TRACKING_USERNAME'] = "annapanfil"
os.environ['MLFLOW_TRACKING_PASSWORD'] = "f3ab7a17321dd66221aae96b7dbc9f43434a812d"

## Conduct experiments

In [ ]:
!python3 src/create_dataset.py --newlines=false

In [ ]:
# sh = """
# #!/bin/bash

# # Function to run training
# run_training() {
#     pretrained_model=$1
#     language=$2
#     epochs=$3
#     command="python src/train.py --pretrained-model=${pretrained_model} --language=${language} --epoch=${epochs} --cuda=True"
#     echo "Running command: $command"
#     $command
# }

# # List of different models to try
# models=("herbert-base" "bert-multiling-uncased" "roberta-multiling" "polish-roberta")
# language="polish"
# epochs=10

# # Run training for each model
# for model in "${models[@]}"
# do
#     run_training "$model" "$language" "$epochs"
# done
# """
# with open('script.sh', 'w') as file:
#   file.write(sh)

# !bash script.sh

In [ ]:
# !python src/train.py --cuda=True --pretrained-model=bert-base-uncased --freeze-bert=False --lstm-dim=-1 --language=english --seed=1 --lr=5e-6 --epoch=8 --use-crf=False --augment-type=all  --augment-rate=0.15 --alpha-sub=0.4 --alpha-del=0.4 --data-path=data --save-path=out
# !python src/train.py --pretrained-model=herbert-base --language=polish --epoch=10

In [ ]:
# !python src/inference.py --pretrained-model=bert-base-uncased --weight-path=out/weights.pt --language=en --in-file=data/test_en.txt --out-file=data/test_en_out.txt
# !python src/inference.py --pretrained-model=herbert-base --weight-path=out/weights.pt --language=pl --in-file=data/test_pl.txt --out-file=data/test_pl_out.txt

In [ ]:
!python src/train.py --freeze-bert=False --lstm=bi --name="Bi, no freeze" --language=polish --cuda=True --pretrained-model=polish-roberta --epoch=10 --log=true
!python src/train.py --freeze-bert=True --lstm=bi --name="Bi, freeze" --language=polish --cuda=True --pretrained-model=polish-roberta --epoch=10 --log=true

!python src/train.py --freeze-bert=False --lstm=uni --name="Uni, no freeze" --language=polish --cuda=True --pretrained-model=polish-roberta --epoch=10 --log=true
!python src/train.py --freeze-bert=True --lstm=uni --name="Uni, freeze" --language=polish --cuda=True --pretrained-model=polish-roberta --epoch=10 --log=true

!python src/train.py --freeze-bert=False --lstm=none --name="No LSTM, no freeze" --language=polish --cuda=True --pretrained-model=polish-roberta --epoch=10 --log=true
!python src/train.py --freeze-bert=True --lstm=none --name="No LSTM, freeze" --language=polish --cuda=True --pretrained-model=polish-roberta --epoch=10 --log=true